In [1]:
!pip install selenium


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Data Scrapping

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from itertools import zip_longest
import pandas as pd
import time

# Initialize WebDriver
page_url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=family"
driver = webdriver.Chrome()
driver.get(page_url)
driver.maximize_window()
time.sleep(2)

# Function to click "Load More" button
def click_load_more():
    try:
        load_more_button = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button/span/span')
        ActionChains(driver).move_to_element(load_more_button).perform()
        load_more_button.click()
        time.sleep(3)  
        return True
    except Exception:
        return False  

# Keep clicking "Load More" until no more content
while click_load_more():
    print("Clicked 'Load More' button.")

print("No more content to load.")

# Extract movie details
movie_names = driver.find_elements(By.CSS_SELECTOR, 'h3.ipc-title__text')
durations = driver.find_elements(By.XPATH, '//*[@class="sc-f30335b4-6 kGhnhC dli-title-metadata"]/span[2]')
ratings = driver.find_elements(By.CLASS_NAME, "ipc-rating-star--rating")
vote_counts = driver.find_elements(By.CLASS_NAME, "ipc-rating-star--voteCount")

# Store extracted data
element_list = []

# Align lists using `zip_longest` to avoid index errors
for name, duration, rating, votes in zip_longest(movie_names, durations, ratings, vote_counts, fillvalue=None):
    element_list.append([
        "Family",
        name.text if isinstance(name, webdriver.remote.webelement.WebElement) else "N/A",
        duration.text if isinstance(duration, webdriver.remote.webelement.WebElement) else "N/A",
        rating.text if isinstance(rating, webdriver.remote.webelement.WebElement) else "N/A",
        votes.text if isinstance(votes, webdriver.remote.webelement.WebElement) else "N/A"
    ])

# Print results
print(element_list)

# Close WebDriver
driver.close()

df = pd.DataFrame(element_list, columns=["Genres","Title", "Duration", "Rating", "Vote Count"])
df.to_csv("imdb_movies_family.csv", index=False)


Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
No more content to load.
[['Family', '1. Flow', '1h 25m', '7.9', ' (55K)'], ['Family', '2. Sonic the Hedgehog 3', '1h 50m', '6.9', ' (51K)'], ['Family', '3. Mufasa: The Lion King', '1h 58m', '6.6', ' (47K)'], ['Family', '4. Despicable Me 4', '1h 34m', '6.2', ' (62K)'], ['Family', '5. Paddington in Peru', '1h 46m', '6.7', ' (16K)'], ['Family', '6. Moana 2', '1h 40m', '6.7', ' (83K)'], ['Family', '7. Inside Out 2', '1h 36m', '7.5', ' (210K)'], ['Family', '8. Wallace & Gromit: Vengeance Most Fowl', '1h 22m', '7.5', ' (32K)'], ['Family', '9. Transformers One', '1h 44m', '7.6', ' (47K)'], ['Family', '10. IF', '1h 44m', '6.4', ' (56K)'], ['Family', '11. The Day the Earth Blew Up: A Looney 

In [ ]:
# Data Cleaning

import pandas as pd
import re

def convert_to_timedelta(duration):
    match = re.match(r'(?:(\d+)h)?\s*(?:(\d+)m)?', duration)  # Extract hours & minutes
    hours = int(match.group(1)) if match.group(1) else 0
    minutes = int(match.group(2)) if match.group(2) else 0
    return pd.Timedelta(hours=hours, minutes=minutes)

def convert_to_numeric(value):
    value = value.strip().upper()  # Ensure consistency (e.g., "k" -> "K")
    if 'K' in value:
        return float(value.replace('K', '')) * 1_000  # Convert 'K' to 1,000
    elif 'M' in value:
        return float(value.replace('M', '')) * 1_000_000  # Convert 'M' to 1,000,000
    else:
        return float(value)  # Convert normal numbers


ad=pd.read_csv('D:\python_vs\datascrap\imdb_movies_family.csv')
ad.dropna(inplace=True)

ad["Title"] = ad["Title"].apply(lambda x: re.sub(r'^\d+\.\s*', '', x))

ad['Duration'] = ad['Duration'].apply(convert_to_timedelta)

ad["Vote Count"]=ad["Vote Count"].str.replace("(","")
ad["Vote Count"]=ad["Vote Count"].str.replace(")","")

ad['Vote Count'] = ad['Vote Count'].apply(convert_to_numeric)

ad["Vote Count"]=ad["Vote Count"].astype(int)


<>:20: SyntaxWarning: invalid escape sequence '\p'
<>:20: SyntaxWarning: invalid escape sequence '\p'
C:\Users\mutha\AppData\Local\Temp\ipykernel_6624\1325363467.py:20: SyntaxWarning: invalid escape sequence '\p'
  ad=pd.read_csv('D:\python_vs\datascrap\imdb_movies_family.csv')


In [70]:
display(ad)

,Genres,Title,Duration,Rating,Vote Count
0,Family,Flow,0 days 01:25:00,7.9,55000
1,Family,Sonic the Hedgehog 3,0 days 01:50:00,6.9,51000
2,Family,Mufasa: The Lion King,0 days 01:58:00,6.6,47000
3,Family,Despicable Me 4,0 days 01:34:00,6.2,62000
4,Family,Paddington in Peru,0 days 01:46:00,6.7,16000
...,...,...,...,...,...
373,Family,Bhabhi Ki Barat,0 days 01:17:00,8.4,6
374,Family,Feng Zi Lao Ba,0 days 01:28:00,5.0,88
375,Family,Haddhu Ledhu Raa,0 days 01:04:00,9.3,86
376,Family,Stealing Church,0 days 02:42:00,8.6,5


In [71]:
ad.info()

<class 'pandas.core.frame.DataFrame'>
Index: 378 entries, 0 to 377
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   Genres      378 non-null    object         
 1   Title       378 non-null    object         
 2   Duration    378 non-null    timedelta64[ns]
 3   Rating      378 non-null    float64        
 4   Vote Count  378 non-null    int64          
dtypes: float64(1), int64(1), object(2), timedelta64[ns](1)
memory usage: 17.7+ KB


In [72]:
ad.to_csv("imdb_movies_family_clean.csv", index=False)

In [ ]:
import pandas as pd

df1=pd.read_csv('D:\python_vs\datascrap\imdb_movies_action_clean.csv')
df2=pd.read_csv('D:\python_vs\datascrap\imdb_movies_adventure_clean.csv')
df3=pd.read_csv('D:\python_vs\datascrap\imdb_movies_animation_clean.csv')
df4=pd.read_csv('D:\python_vs\datascrap\imdb_movies_family_clean.csv')
df5=pd.read_csv('D:\python_vs\datascrap\imdb_movies_mystery_clean.csv')

final_data=pd.concat([df1,df2,df3,df4,df5],axis=0)

final_data["Duration"]=final_data["Duration"].str.replace("0 days","")

display(final_data)



<>:3: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:5: SyntaxWarning: invalid escape sequence '\p'
<>:6: SyntaxWarning: invalid escape sequence '\p'
<>:7: SyntaxWarning: invalid escape sequence '\p'
<>:3: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:5: SyntaxWarning: invalid escape sequence '\p'
<>:6: SyntaxWarning: invalid escape sequence '\p'
<>:7: SyntaxWarning: invalid escape sequence '\p'
C:\Users\mutha\AppData\Local\Temp\ipykernel_9480\1832326806.py:3: SyntaxWarning: invalid escape sequence '\p'
  df1=pd.read_csv('D:\python_vs\datascrap\imdb_movies_action_clean.csv')
C:\Users\mutha\AppData\Local\Temp\ipykernel_9480\1832326806.py:4: SyntaxWarning: invalid escape sequence '\p'
  df2=pd.read_csv('D:\python_vs\datascrap\imdb_movies_adventure_clean.csv')
C:\Users\mutha\AppData\Local\Temp\ipykernel_9480\1832326806.py:5: SyntaxWarning: invalid escape sequence '\p'
  df3=pd.read_csv('D:

,Genres,Title,Duration,Rating,Vote Count
0,Action,Dune: Part Two,02:46:00,8.5,610000
1,Action,Venom: The Last Dance,01:50:00,6.0,105000
2,Action,Gladiator II,02:28:00,6.6,214000
3,Action,Sonic the Hedgehog 3,01:50:00,6.9,51000
4,Action,The Lord of the Rings: The War of the Rohirrim,02:14:00,6.3,27000
...,...,...,...,...,...
428,Mystery,The Director's Cut,01:54:00,8.6,21
429,Mystery,Churul,00:11:00,2.6,7
430,Mystery,Creatures of Habit,01:39:00,8.2,6
431,Mystery,Dream Thief,01:06:00,6.6,6


In [3]:
!pip install mysql-connector-python


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import mysql.connector

connection = mysql.connector.connect(
  host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
  port = 4000,
  user = "kCCeTyfqG4q97x6.root",
  password = "O5K4JarXblpcn7gg",
  database = "testing",

)

mycursor = connection.cursor(buffered=True)


In [5]:
mycursor.execute("create database imdb")

In [ ]:
mycursor.execute("create table imdb.movies_2024 (genres varchar(255), title varchar(255), duration time, rating float, vote_count int) ")



In [ ]:
insert_query = "INSERT INTO imdb.movies_2024 (genres,title, duration, rating, vote_count) VALUES (%s, %s,%s, %s,%s)"
values = final_data[['Genres', 'Title','Duration','Rating','Vote Count']].values.tolist()   

mycursor.executemany(insert_query, values)  
connection.commit()  

print("Data inserted into Database successfully!")

Data inserted into TiDB successfully!
